Once you complete the fit you will find two files one starting with (in our case) with "newtrial", which contains the fit information and the other file with "fittoy" which contains the toy events after the fit is done. 

In [ ]:
import warnings
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Set warnings to ignore performance warnings in Pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

bins = 100
range_min, range_max = data_df['mdpluspim122'].min(), data_df['mdpluspim122'].max()
bin_width = (range_max - range_min) / bins

# Load the fit_toy data
fit_toy = uproot.open("fittoy0_expt0.root")["genResults;1"].arrays(library="pd")

plt.figure(figsize=(10, 6))
(data_df['mdpluspim122']**0.5).hist(bins=bins, weights=data_df['SigWeight'], density=True, label='sweighted applied input')
fit_toy['m13'].hist(bins=bins, histtype='step', density=True, color='black', label='Fit-toy')

plt.xlabel(r'$m_{D \pi}^{max}$ [GeV]', fontsize=18)
plt.ylabel(f'Entries / {bin_width:.4f} GeV', fontsize=18)
plt.minorticks_on()
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tick_params(axis='both', which='minor', labelsize=10)
plt.legend(fontsize=13)
plt.grid()
plt.show()

In [ ]:
for i in range(1):
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

    fit = uproot.open(f"newtrial_{i}_expt_0-0.root")["fitResults;1"]
    fit_data = fit.arrays(library="pd")
    
    x = []
    y = []
    complex_points = []
    
    for i in range(16):
        a_key = f'Dst0_0_Re{i}'
        d_key = f'Dst0_0_Im{i}'
#The following if statements are used because LAURA++ does not store the ampltiude and phase of the fixed knots.
        if i == 3:
            x.append(0.5)
            y.append(0.0)
        elif i == 12:
            x.append(0.0)
            y.append(0.0)
        else:
            if a_key in fit_data.columns:
                x.append(fit_data[a_key].values[0])  
            if d_key in fit_data.columns:
                y.append(fit_data[d_key].values[0])
    
    # Convert lists to numpy arrays
    x = np.array(x)
    y = np.array(y)

    m = 15
    complex_points = x[:m] + 1j * y[:m]
    t = np.arange(len(complex_points))  # Parameter t along the spline
    
    # Fit cubic splines to both real and imaginary parts separately
    spline_real = CubicSpline(t, x[:m], bc_type='not-a-knot')
    spline_imag = CubicSpline(t, y[:m], bc_type='not-a-knot')
    
    t_dense = np.linspace(t.min(), t.max(), 300)
    
    real_dense = spline_real(t_dense)
    imag_dense = spline_imag(t_dense)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, color='black', s=50, zorder=3)
    
    for idx, (re, im) in enumerate(zip(x, y)):
        plt.text(re, im, f' {idx}', fontsize=9, ha='left', va='bottom', zorder=4)
    
    plt.plot(real_dense, imag_dense, 'r.', label='Cubic Spline Fit', zorder=2)
    
    plt.xlabel('Re', fontsize=18)
    plt.ylabel('Im', fontsize=18)
    plt.title('Argand Plot', fontsize=20)
    plt.legend()
    plt.grid(True)
    plt.show()   
    print(fit_data['NLL'])



